In [ ]:
import torch
import torch.nn.functional as F
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn.functional as F

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, inp, out):
            self.activations = out.detach()

        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def __call__(self, x):
        self.model.zero_grad()
        output = self.model(x)

        if isinstance(output, tuple):
            output = output[0]

        class_idx = output.argmax(dim=1).item()
        score = output[:, class_idx]
        score.backward()

        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1)

        cam = F.relu(cam)
        cam = cam.squeeze().cpu().numpy()
        cam -= cam.min()
        cam /= cam.max() + 1e-8

        return cam


In [ ]:
import torch
import torch.nn as nn
from torchvision import models

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inception = models.inception_v3(
    weights=None,
    aux_logits=True
)

num_classes = len(train_dataset_inception.classes)

inception.fc = nn.Sequential(
    nn.Linear(inception.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)

# Load trained weights
inception.load_state_dict(
    torch.load("inception_best.pth", map_location=DEVICE)
)

inception = inception.to(DEVICE)
inception.eval()

# Grad-CAM setup
grad_cam = GradCAM(
    model=inception,
    target_layer=inception.Mixed_7c
)

print("InceptionV3 loaded (Grad-CAM ready)")

In [ ]:
import matplotlib.pyplot as plt

comparison_df.set_index("Model")[[
    "Accuracy",
    "F1-score (macro)"
]].plot(
    kind="bar",
    figsize=(8,5),
    title="Comparison of CNN Architectures on Test Set"
)

plt.ylabel("Score")
plt.ylim(0, 1)
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inception = models.inception_v3(
    weights=None,
    aux_logits=True
)

num_classes = len(train_dataset_inception.classes)

inception.fc = nn.Sequential(
    nn.Linear(inception.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, num_classes)
)

# Load trained weights
inception.load_state_dict(
    torch.load("inception_best.pth", map_location=DEVICE)
)

inception = inception.to(DEVICE)
inception.eval()

# Grad-CAM setup
grad_cam = GradCAM(
    model=inception,
    target_layer=inception.Mixed_7c
)

print("InceptionV3 loaded (Grad-CAM ready)")

In [ ]:
import matplotlib.pyplot as plt
import cv2
import random

inception.eval()

samples = random.sample(range(len(test_dataset_inception)), 5)

for idx in samples:
    image, label = test_dataset_inception[idx]

    input_tensor = image.unsqueeze(0).to(DEVICE)

    cam = grad_cam(input_tensor)

    img = image.permute(1, 2, 0).cpu().numpy()
    img = (img - img.min()) / (img.max() - img.min())

    cam_resized = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255

    overlay = heatmap * 0.4 + img

    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(cam_resized, cmap="jet")
    plt.title("Grad-CAM")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    plt.title("Overlay")
    plt.axis("off")

    plt.show()